<a href="https://colab.research.google.com/github/Yang-star-source/Latent_Diffusion_From_Scratch/blob/main/CLIP_EMBEDDING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import hf_hub_download
import os

repo_id = "ziyang06315/cats_images_dataset"
CAPTIONS_PATH = hf_hub_download(repo_id=repo_id, filename="captions.zip", repo_type="dataset")
CAPTION_DIR = "/content/captions"
os.makedirs(CAPTION_DIR, exist_ok=True)
os.system(f"unzip -q {CAPTIONS_PATH} -d {CAPTION_DIR}")

In [ ]:
import torch
import os
from transformers import CLIPTextModel , CLIPTokenizer

CAPTIONS_PATH = "/content/captions/captions"
EMBEDDINGS_PATH = "/content/captions/embeddings"
os.makedirs(EMBEDDINGS_PATH, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "openai/clip-vit-large-patch14"
tokenizer = CLIPTokenizer.from_pretrained(model_id)
text_encoder = CLIPTextModel.from_pretrained(model_id).to(DEVICE)
text_encoder.eval()

files = os.listdir(CAPTIONS_PATH)
print("Start Processing")
for file in files:
    with open(os.path.join(CAPTIONS_PATH, file), "r") as f:
        text = f.read().strip()

    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", padding="max_length", max_length=77, truncation=True).to(DEVICE)
        embedding = text_encoder(**inputs).last_hidden_state.cpu()[0]

    torch.save(embedding, os.path.join(EMBEDDINGS_PATH, file.replace(".txt", ".pt")))

# Always shape [77,768]
print(f"Embedding shape : {embedding.shape}")
print("Finished")


In [ ]:
import shutil

save_path = "/content/captions_embedding"
folder_to_zip = "/content/captions/embeddings"
shutil.make_archive(save_path, "zip", folder_to_zip)
print("Finish zipping")

In [ ]:
from google.colab import files

# Downloadding , a bit slow , use google drive if need fast
files.download("/content/captions_embedding.zip")